[![Fixel Algorithms](https://fixelalgorithms.co/images/CCExt.png)](https://fixelalgorithms.gitlab.io)

# Machine Learning Methods

## Supervised Learning - Classification - K-NN Classifier

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 0.1.000 | 20/01/2023 | Royi Avital | First version                                                      |
|         |            |             |                                                                    |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/MachineLearningMethods/2023_01/0009ClassifierKnn.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

# Machine Learning
from sklearn.datasets import load_breast_cancer, make_circles
from sklearn.neighbors import KNeighborsClassifier

# Miscellaneous
import os
from platform import python_version
import random

# Typing
from typing import Tuple

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Jupyter
from IPython import get_ipython
from IPython.display import Image, display
from ipywidgets import Dropdown, FloatSlider, interact, IntSlider, Layout

## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

In [ ]:
# Configuration
%matplotlib inline

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# sns.set_theme() #>! Apply SeaBorn theme

runInGoogleColab = 'google.colab' in str(get_ipython())

In [ ]:
# Constants

FIG_SIZE_DEF = (8, 8)
ELM_SIZE_DEF = 50
CLASS_COLOR = ('b', 'r')


In [ ]:
# Fixel Algorithms Packages


In [ ]:
# Parameters

# Data Generation
numCircles0 = 250
numCircles1 = 250
numSwaps    = 50 #<! Number of samples to swap between inner circle and outer circle
noiseLevel  = 0.03

# Data Visualization
elmSize     = 50
classColor0 = 'b'
classColor1 = 'r'

numGridPts = 250

In [ ]:
# Auxiliary Functions

def PlotBinaryClassData( mX: np.ndarray, vY: np.ndarray, hA:plt.Axes = None, figSize: Tuple[int, int] = FIG_SIZE_DEF, elmSize: int = ELM_SIZE_DEF, classColor: Tuple[str, str] = CLASS_COLOR, axisTitle: str = None ) -> plt.Axes:

    if hA is None:
        hF, hA = plt.subplots(figsize = figSize)
    else:
        hF = hA.get_figure()
    
    vC, vN = np.unique(vY, return_counts = True)

    numClass = len(vC)
    if (len(vC) != 2):
        raise ValueError(f'The input data is not binary, the number of classes is: {numClass}')

    vIdx0 = vY == vC[0]
    vIdx1 = vY == vC[1] #<! Basically ~vIdx0

    hA.scatter(mX[vIdx0, 0], mX[vIdx0, 1], s = elmSize, color = classColor[0], edgecolor = 'k', label = f'$C_\u007b {vC[0]} \u007d$')
    hA.scatter(mX[vIdx1, 0], mX[vIdx1, 1], s = elmSize, color = classColor[1], edgecolor = 'k', label = f'$C_\u007b {vC[1]} \u007d$')
    hA.axvline(x = 0, color = 'k')
    hA.axhline(y = 0, color = 'k')
    hA.axis('equal')
    if axisTitle is not None:
        hA.set_title(axisTitle)
    hA.legend()
    
    return hA

## Generate / Load Data


In [ ]:
# Loading / Generating Data
numCircles = numCircles0 + numCircles1
mX, vY     = make_circles((numCircles0, numCircles1), shuffle = False, noise = noiseLevel, random_state = seedNum)

print(f'The features data shape: {mX.shape}')
print(f'The labels data shape: {vY.shape}')

In [ ]:
# Swapping some samples between the classes
# The first numCircles0 are for class 0
vSwapIdx = np.random.choice(numCircles0, numSwaps, replace = False)
vY[vSwapIdx] = 1
vSwapIdx = numCircles0 + np.random.choice(numCircles1, numSwaps, replace = False)
vY[vSwapIdx] = 0

### Plot Data

In [ ]:
# Display the Data

hA = PlotBinaryClassData(mX, vY)

## Train a K-NN Classifier

The K-NN classifier, given a new sample $\boldsymbol{x}_{i}$ basically do as following:

1. Find the `K` nearest samples (By the chosen distance function) to the given point.
2. Build an histogram of the classes of the `K` points.
3. Set the class of $\boldsymbol{x}_{i}$ to be the $\arg \max$ of the histogram.

<font color='red'>(**?**)</font> In case there is non unique $\arg \max$ in (3), what should be done?  
<font color='red'>(**?**)</font> Can you think on alternative decision rules for step (3)?  

In [ ]:
# Grid of the data support
v0       = np.linspace(mX[:, 0].min() - 0.1, mX[:, 0].max() + 0.1, numGridPts)
v1       = np.linspace(mX[:, 1].min() - 0.1, mX[:, 1].max() + 0.1, numGridPts)
XX0, XX1 = np.meshgrid(v0, v1)
XX       = np.c_[XX0.ravel(), XX1.ravel()]

def PlotKnn( K ):
    # Train the a K-NN classifier
    oKnnClassifier = KNeighborsClassifier(n_neighbors = K, p = 2).fit(mX, vY) #<! Training on the data
    
    # Plot classification
    Z = oKnnClassifier.predict(XX) #<! Prediction on the grid (The support)
    Z = Z.reshape(XX0.shape)

    hF, hA = plt.subplots(figsize = (8, 8))
    hA = PlotBinaryClassData(mX, vY, hA = hA, axisTitle = f'K-NN Classifier - $K = {K}$')
    hA.contourf(XX0, XX1, Z, colors = [classColor0, classColor1], alpha = 0.3, levels = [-0.5, 0.5, 1.5])

In [ ]:
# Display the Geometry of the Classifier

kSlider = IntSlider(min = 1, max = 21, step = 2, value = 1, layout = Layout(width = '30%'))
interact(PlotKnn, K = kSlider)

plt.show()